# Agentes com Autogen - Function Calling - Hands-On

Agradecimento ao professor Julio Cesar dos Reis pelo material.

## Descrição

Exploramos a implementação de agentes simples utilizando a biblioteca Autogen. O objetivo é demonstrar como configurar e interagir com agentes capazes de executar tarefas automatizadas de forma eficiente. Aprenderemos a estruturar prompts, configurar agentes com diferentes especializações e testar interações dinâmicas entre eles.


In [1]:
!pip install autogen
!pip install --upgrade pydantic
!pip install ipywidgets
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 651.1/651.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.4 MB/s eta 0:00:00


In [2]:
from typing import Literal
import autogen
from typing_extensions import Annotated
import getpass

In [8]:
# Solicitar a API key de forma segura
api_key = getpass.getpass(prompt='Digite sua API Key: ')

config_list = [
    {
      "model": "llama3-8b-8192",
      "api_key": api_key,
      "api_type": "groq"
    }
]

Digite sua API Key: ··········


Uma ferramenta deve ser registrada com pelo menos dois agentes para que seja útil na conversação. O agente registrado com a assinatura da ferramenta por meio de register_for_llm pode chamar a ferramenta; o agente registrado com o objeto de função da ferramenta por meio de register_for_execution pode executar a função da ferramenta.

In [13]:
llm_config = {
    "config_list": config_list,
    "timeout": 120
}

#agente que efetua conversão de moeda
currency_bot = autogen.AssistantAgent(
    name="currency_bot",
    system_message="Para tarefas de troca de moeda, use apenas as funções que você recebeu. Responda TERMINATE "
                   "quando a tarefa estiver concluída.",
    llm_config=llm_config
)

#agente user proxy
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    code_execution_config=False
)

CurrencySymbol = Literal["USD", "EUR", "BTC"]

#função de taxa_de_câmbio
def exchange_rate(base_currency: CurrencySymbol, quote_currency: CurrencySymbol) -> float:
    if base_currency == quote_currency:
        return 1.0
    elif base_currency == "USD" and quote_currency == "EUR":
        return 1 / 1.09
    elif base_currency == "EUR" and quote_currency == "USD":
        return 1 / 1.1
    elif base_currency == "USD" and quote_currency == "BTC":
        return 1 / 90777
    elif base_currency == "EUR" and quote_currency == "BTC":
        return 1 / 84120
    else:
        raise ValueError(f"Moedas desconhecidas: {base_currency}, {quote_currency}")  # Tradução: "Moedas desconhecidas:..."

#registrando a ferramenta
@user_proxy.register_for_execution()
# Register the tool signature with the assistant agent.
@currency_bot.register_for_llm(description="Calculadora de câmbio de moeda")  # Tradução: "Calculadora de câmbio de moeda"
def currency_calculator(
        base_amount: Annotated[float, "Quantia da moeda em base_currency"],  # Tradução: "Quantia da moeda em base_currency"
        base_currency: Annotated[CurrencySymbol, "Moeda base"] = "USD",  # Tradução: "Moeda base"
        quote_currency: Annotated[CurrencySymbol, "Moeda de cotação"] = "EUR"  # Tradução: "Moeda de cotação"
) -> str:
    quote_amount = exchange_rate(base_currency, quote_currency) * base_amount
    return f"{quote_amount} - {quote_currency}"


#######################################
#inicialização da conversa
user_proxy.initiate_chat(
    currency_bot,
    message="Converter 1000 EUR para BTC"  # Tradução: "Converter 100 USD para EUR"
)


user_proxy (to currency_bot):

Converter 1000 EUR para BTC

--------------------------------------------------------------------------------
currency_bot (to user_proxy):

***** Suggested tool call (call_88rc): currency_calculator *****
Arguments: 
{"base_amount":1000,"base_currency":"EUR","quote_currency":"BTC"}
****************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION currency_calculator...
Call ID: call_88rc
Input arguments: {'base_amount': 1000, 'base_currency': 'EUR', 'quote_currency': 'BTC'}
user_proxy (to currency_bot):

***** Response from calling tool (call_88rc) *****
0.011887779362815026 - BTC
**************************************************

--------------------------------------------------------------------------------
currency_bot (to user_proxy):

***** Suggested tool call (call_e2g4): currency_calculator *****
Arguments: 
{"base_amount":0.011887779362

ChatResult(chat_id=None, chat_history=[{'content': 'Converter 1000 EUR para BTC', 'role': 'assistant', 'name': 'user_proxy'}, {'tool_calls': [{'id': 'call_88rc', 'function': {'arguments': '{"base_amount":1000,"base_currency":"EUR","quote_currency":"BTC"}', 'name': 'currency_calculator'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': '0.011887779362815026 - BTC', 'tool_responses': [{'tool_call_id': 'call_88rc', 'role': 'tool', 'content': '0.011887779362815026 - BTC'}], 'role': 'tool', 'name': 'user_proxy'}, {'tool_calls': [{'id': 'call_e2g4', 'function': {'arguments': '{"base_amount":0.011887779362815026,"base_currency":"BTC","quote_currency":"USD"}', 'name': 'currency_calculator'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}], summary='', cost={'usage_including_cached_inference': {'total_cost': 0.00012434000000000002, 'llama3-8b-8192': {'cost': 0.00012434000000000002, 'prompt_tokens': 2250, 'completion_tokens': 148, 'total_tokens': 2398}}, 'us